# 实验六：决策树分类器
- 姓名：吴静
- 学号：2113285
- 班级：信息安全二班

## 实验要求

### 基本要求
- 基于 Watermelon-train1数据集（只有离散属性），构造ID3决策树；
- 基于构造的 ID3 决策树，对数据集 Watermelon-test1进行预测，输出分类精度；

### 中级要求
- 对数据集Watermelon-train2，构造C4.5或者CART决策树，要求可以处理连续型属性；
- 对测试集Watermelon-test2进行预测，输出分类精度；

### 高级要求
使用任意的剪枝算法对构造的决策树（基本要求和中级要求构造的树）进行剪枝，观察测试集合的分类精度是否有提升，给出分析过程。

## 导入需要的包

In [34]:
import pandas as pd
import numpy as np
import copy
import math

## ID3

### 导入数据集

In [35]:
def readfile(filename):
    
    #读取csv文件，选择中文字符
    data=pd.read_csv(filename,encoding="gb2312",header=None)
    
    #取读取中的文件中的第二行到最后一行（第一行是标识符），第二列到最后一列（第一列是数字）
    #value取内容后tolist()转化为列表类型
    datacontent=data.iloc[1:,1:].values.tolist()
    
    datafeature=data.iloc[0,1:].values.tolist()
    #返回
    return datacontent,datafeature

In [36]:
train1,feature_train1=readfile("Watermelon-train1.csv")
test1,feature_test1=readfile("Watermelon-test1.csv")

In [37]:
print(train1)

[['青绿', '蜷缩', '浊响', '清晰', '是'], ['乌黑', '蜷缩', '沉闷', '清晰', '是'], ['乌黑', '蜷缩', '浊响', '清晰', '是'], ['青绿', '蜷缩', '沉闷', '清晰', '是'], ['浅白', '蜷缩', '浊响', '清晰', '是'], ['青绿', '稍蜷', '浊响', '清晰', '是'], ['乌黑', '稍蜷', '浊响', '稍糊', '是'], ['乌黑', '稍蜷', '浊响', '清晰', '是'], ['乌黑', '稍蜷', '沉闷', '稍糊', '否'], ['青绿', '硬挺', '清脆', '清晰', '否'], ['浅白', '硬挺', '清脆', '模糊', '否'], ['浅白', '蜷缩', '浊响', '模糊', '否'], ['青绿', '稍蜷', '浊响', '稍糊', '否'], ['浅白', '稍蜷', '沉闷', '稍糊', '否'], ['浅白', '蜷缩', '浊响', '模糊', '否'], ['青绿', '蜷缩', '沉闷', '稍糊', '否']]


### 信息熵

- $$ H(X)=-\sum_{i=1}^{n} p\left(x_{i}\right) \log p\left(x_{i}\right)$$ 

$𝐻(𝑋)$就被称作随机变量𝑋的熵，它表示随机变量不确定的度量。熵取值越大，随机变量不确定性越大。当随机变量为均匀分布时，熵最大。当某一状态概率取值为1时，熵的值为零。

- value_counts(values,sort=True, ascending=False, normalize=False,bins=None,dropna=True)
  - values：要统计的列
  - sort：是否进行排序
  - asscending：若为True，则是按出现次数从高到低排序
  - normalize：是否进行标准化

In [38]:
def H_X(data):
    #需要分割出最后一列进行计数
    #使用iloc函数，于是需要先转化成df类型
    #设置normalize进行标准化
    #默认标签在最后一列
    a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
    #将（series）a的索引（即类别值）转化成列表
    #alist=a.index.tolist()
    #返回累加后的值
    return sum(np.log2(a) * a * (-1))

### 信息增益 $𝐻(𝐷∣𝑋)$ ：
- 条件熵 $𝐻(𝑌∣𝑋)$ ：
	表示在已知随机变量𝑋的条件下随机变量𝑌的不确定性，定义为给定𝑋条件下𝑌的条件概率分布的熵对𝑋的数学期望:
$$H(Y \mid X)=\sum_{x} p(x) H(Y \mid X=x) =-\sum_{x} p(x) \sum_{y} p(y \mid x) \log p(y \mid x)$$

- 特征𝐴对数据集𝐷的信息增益就是熵$𝐻(𝐷)$与条件熵$𝐻(𝐷|𝐴)$之差:
$$𝐻(𝐷)−𝐻(𝐷∣𝐴)$$

表示已知特征𝐴的信息而使得数据集𝐷的信息不确定减少的程度。信息增益越大的特征代表其具有更强的分类能力，所以我们就要**选择能够使数据的不确定程度减少最多的特征**，也就是信息增益最大的特征。

In [39]:
def g_D_A(data,feature,A,D=4):

    #转换为df类型，特征存储在feature中
    df=pd.DataFrame(data,columns=[feature[i] for i in range(len(feature))])

    #根据A进行分类
    grouped=df.groupby(df.columns[A])

    print("特征为：",feature[A])
    for group, values in grouped:
        print(f"Category: {group}, Values: {values.to_dict(orient='records')}")
    
    #计算H_Dn
    grouped_data=grouped.apply(lambda x:H_X(x))

    #计算H_Dn前面的系数
    p=pd.value_counts(df.iloc[:,A],normalize=True)

    #计算H_D_A
    H_D_A=sum(p*grouped_data)

    #计算H_D
    H_D=H_X(data)
    
    #计算g_D_A
    result=H_D-H_D_A

    return result


In [40]:
g_D_A(train1,feature_train1,0,4)

特征为： 色泽
Category: 乌黑, Values: [{'色泽': '乌黑', '根蒂': '蜷缩', '敲声': '沉闷', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '稍糊', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '沉闷', '纹理': '稍糊', '好瓜': '否'}]
Category: 浅白, Values: [{'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '浅白', '根蒂': '硬挺', '敲声': '清脆', '纹理': '模糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '模糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '稍蜷', '敲声': '沉闷', '纹理': '稍糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '模糊', '好瓜': '否'}]
Category: 青绿, Values: [{'色泽': '青绿', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '蜷缩', '敲声': '沉闷', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '硬挺', '敲声': '清脆', '纹理': '清晰', '好瓜': '否'}, {'色泽': '青绿', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '稍糊', '好瓜': '否'}, {'色泽': '

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\2337398345.py:17: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Ser

0.17379494069539858

### 选择最优特征

In [41]:
def chooseBestFeature(data,feature,D):
    if type(data)!=list:
        data=data.values.tolist()
    if type(feature)!=list:
        feature=feature.values.tolist()
        
    best=0.0
    bestindex=-1

    #data[0]中还含有“好瓜”的标签，需要剔除
    #遍历所有特征(列)
    for i in range(len(data[0])-1):
        temp=g_D_A(data,feature,i,D)
        print("g_D_A=",temp)
        print("\n")
        if temp>best:
            best=temp
            bestindex=i
    
    return bestindex

In [42]:
chooseBestFeature(train1,feature_train1,4)

特征为： 色泽
Category: 乌黑, Values: [{'色泽': '乌黑', '根蒂': '蜷缩', '敲声': '沉闷', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '稍糊', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '沉闷', '纹理': '稍糊', '好瓜': '否'}]
Category: 浅白, Values: [{'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '浅白', '根蒂': '硬挺', '敲声': '清脆', '纹理': '模糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '模糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '稍蜷', '敲声': '沉闷', '纹理': '稍糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '模糊', '好瓜': '否'}]
Category: 青绿, Values: [{'色泽': '青绿', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '蜷缩', '敲声': '沉闷', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '硬挺', '敲声': '清脆', '纹理': '清晰', '好瓜': '否'}, {'色泽': '青绿', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '稍糊', '好瓜': '否'}, {'色泽': '

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\2337398345.py:17: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Ser

3

### 构建决策树

- 决策树的生成:

	从根节点开始，计算所有可能特征的信息增益，选择信息增益最大的特征作为划分该节点的特征，根据该特征的不同取值建立子节点；
	在对子节点递归地调用以上方法，直到达到停止条件，得到⼀个决策树。
    
    
- 迭代停止条件：
  1. 当前结点所有样本都属于同⼀类别；
  2. 当前结点的所有属性值都相同，没有剩余属性可用来进一步划分样本；
  3. 达到最大树深；
  4. 达到叶子结点的最小样本数；

In [43]:
def decision_tree(X, y, features,D):
    #都属于同一类别
    if len(set(y)) == 1:
        return y.iloc[0]
    
    #没有剩余属性可以用来划分样本
    if len(features) == 0:
        #idxmax()返回具有最大值的索引
        return y.value_counts().idxmax()
    
    #如果还有多个类别
    #找到最优划分的下标
    bestindex=chooseBestFeature(X,features,D)
    #找到最优划分的特征值
    best_feature=features[bestindex]
    #构建决策树
    tree = {best_feature: {}}
    #最优特征值已经被选择，之后不再进行选择，所以进行剔除
    remaining_features = [feature for feature in features if feature != best_feature]
    
    #对于最佳特征列中的每一个值
    for value in X[best_feature].unique():
        #取出包含当前值的行
        subset_X = X[X[best_feature] == value]
        #由于该特征值已经使用过，此时删除
        subset_X=subset_X.drop(best_feature,axis=1)
        #根据subset_X的索引值获取相应的y的值
        subset_y = y.loc[subset_X.index]
        #递归调用decision_tree函数
        tree[best_feature][value] = decision_tree(subset_X, subset_y, remaining_features,D)
    return tree

In [44]:
df=pd.DataFrame(train1,columns=[feature_train1[i] for i in range(len(feature_train1))])

#X =df.drop("好瓜",axis=1)  # 获取特征
#特征+标签
X=df

#标签
y = df['好瓜']

In [45]:
tree=decision_tree(X,y,feature_train1,4)

特征为： 色泽
Category: 乌黑, Values: [{'色泽': '乌黑', '根蒂': '蜷缩', '敲声': '沉闷', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '稍糊', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '乌黑', '根蒂': '稍蜷', '敲声': '沉闷', '纹理': '稍糊', '好瓜': '否'}]
Category: 浅白, Values: [{'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '浅白', '根蒂': '硬挺', '敲声': '清脆', '纹理': '模糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '模糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '稍蜷', '敲声': '沉闷', '纹理': '稍糊', '好瓜': '否'}, {'色泽': '浅白', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '模糊', '好瓜': '否'}]
Category: 青绿, Values: [{'色泽': '青绿', '根蒂': '蜷缩', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '蜷缩', '敲声': '沉闷', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '清晰', '好瓜': '是'}, {'色泽': '青绿', '根蒂': '硬挺', '敲声': '清脆', '纹理': '清晰', '好瓜': '否'}, {'色泽': '青绿', '根蒂': '稍蜷', '敲声': '浊响', '纹理': '稍糊', '好瓜': '否'}, {'色泽': '

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\2337398345.py:17: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Ser

In [46]:
print(tree)

{'纹理': {'清晰': {'根蒂': {'蜷缩': '是', '稍蜷': '是', '硬挺': '否'}}, '稍糊': {'色泽': {'乌黑': {'敲声': {'浊响': '是', '沉闷': '否'}}, '青绿': '否', '浅白': '否'}}, '模糊': '否'}}


### 分类

In [47]:
# 对单个样本进行分类
def classify(node, sample):
    if isinstance(node, dict):
        key = list(node.keys())[0]
        value = sample[key]
        #print("value",value)
        if value in node[key]:
            return classify(node[key][value], sample)
        else:
            return "Unknown"
    else:
        return node

### 预测

In [48]:
# 对测试集进行预测
def predict(tree, test_data):
    predictions = []
    for index, row in test_data.iterrows():
        prediction = classify(tree, row)
        predictions.append(prediction)
    return predictions

In [49]:
test_data = pd.read_csv('Watermelon-test1.csv',encoding="gb2312") 

In [50]:
predictions = predict(tree, test_data)
print(predictions)

['是', '是', '是', '否', '是', '是', '是', '否', '否', '否']


### 计算精度

In [51]:
def simrate(data, predict):
    num = 0
    for i in range(len(data)):
        if data[i][-1] == predict[i]:
            num +=1
    return format(num / len(data), '.2%')

In [52]:
result=simrate(test_data["好瓜"],predictions)
print("ID3分类器对于test1数据集的准确率是：", result)

ID3分类器对于test1数据集的准确率是： 70.00%


## C4.5

### 导入数据集

由于这里面用的方法与ID3的不同，所以这里重新导入数据集，选择此时需要的特征与数据。

In [53]:
def readfile1(filename):
    df = pd.read_csv(filename, encoding = 'gb2312')
    temp = np.array(df).tolist()
    for i in temp:
        i.pop(0)
    return temp

In [54]:
train2 = readfile1("Watermelon-train2.csv")
test2 = readfile1("Watermelon-test2.csv")
labels2 = ["色泽", "根蒂", "敲声", "纹理", "密度", "好瓜"]

### 信息增益比

In [55]:
def gr(data,feature,A,D=4):
    #计算固有值intrinsic_value
    intrinsic_value=0
    #print(data)
    #转换为df类型，特征存储在feature中
    df=pd.DataFrame(data,columns=[feature[i] for i in range(len(feature))])
    feature_class=df[feature[A]].value_counts()
    for i in feature_class.keys():
        weight=feature_class[i]/len(data)
        intrinsic_value+=-weight*np.log2(weight)
    return g_D_A(data,feature,A,D=4)/intrinsic_value

### 删除行

#### 删除离散型的行

In [56]:
def split(data, index, kind):
    ls = []
    for temp in data:
        if temp[index] == kind:
            t = temp[0: index]
            t = t + temp[index + 1: ]
            ls.append(t)
    return ls

#### 删除连续型的行

In [57]:
def split2(data, index, kind, method):
    ls = []
    if method == 0:
        for temp in data:
            if temp[index] <= kind:
                t = temp[0 : index]
                t = t + temp[index + 1 : ]
                ls.append(t)
    else:
        for temp in data:
            if temp[index] > kind:
                t = temp[0 : index]
                t = t + temp[index + 1 : ]
                ls.append(t)
    return ls

### 选择最优特征

In [58]:
def chooseBestFeatureToSplit2(data):
    #原始的信息熵
    base= H_X(data)
    #存储每个特征列的信息熵
    info = []
    for j in range(len(data[0]) - 1):
        #用字典存储特征值及其出现次数
        dic = {}
        for i in data:
            #取出最后的结果，获取当前特征列的值
            current = i[j]
            if current not in dic.keys(): 
                #创建一个新的类别
                dic[current] = 1
            else:
                #原有类别+1
                dic[current] += 1
        result = 0.0
        for key in dic:
            #计算特征值出现的概率
            prob = float(dic[key]) / len(data)
            #计算信息熵
            result -= prob * math.log(prob,2) 
        info.append(result)
    
    #最佳信息增益
    best = 0
    #最佳特征索引
    bestindex = -1
    #最佳分割点的值
    bestpartvalue = None 

    #如果是离散值，使用该值进行分割
    for i in range(len(data[0]) - 1):
        #抽取该列数据的所有信息
        ls = [index[i] for index in data]
        feture = set(ls)

        #计算该列的信息增益
        temp = 0.0
        #print(type(ls[0]))
        #判断是否时离散的
        if type(ls[0]) == type("a"):
            for value in feture:
                #根据特征值进行数据集分割
                datatemp = split(data, i, value)
                #计算分割后数据集的概率
                prob = len(datatemp) / float(len(data))
                #计算分割后数据集的信息熵
                t = H_X(datatemp)
                #计算加权信息熵
                temp += prob * t
        else:
            #对连续值特征列进行排序
            ls.sort()
            min = float("inf")
            for j in range(len(ls) - 1):
                #计算划分点
                part = (ls[j + 1] + ls[j]) / 2
                #左子集数据
                left = split2(data, i, part, 0)
                #右子集数据
                right = split2(data, i, part, 1)

                #左子集的信息熵
                temp1 = H_X(left)
                #右子集的信息熵
                temp2 = H_X(right)
                #加权信息熵
                temp = len(left) / len(data) * temp1 + len(right) / len(data) * temp2
                if temp < min:
                    min = temp
                    bestpartvalue = part
            #更新信息增益
            temp = min
        infoGain = base - temp

        #根据信息增益挑选 
        if info[i] != 0:
            if infoGain / info[i] >= best:
                best = infoGain / info[i]
                bestindex = i
    
    return bestindex, bestpartvalue


In [59]:
index1,partvalue1=chooseBestFeatureToSplit2(train2)
print(index1)

3


C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Seri

### 构建决策树

In [60]:
def decision_tree1(data, labels):
    df=pd.DataFrame(data,columns=[labels[i] for i in range(len(labels))])
    #print(len(df["好瓜"].unique()))
    #取出该数据集的分类
    typelist = [index[-1] for index in df]
    #计算出类别种类以及数量
    typecount = len(df["好瓜"].unique())
    if typecount == len(typelist):
        #如果只有一个类别
        return typelist[0]

    #最优划分属性的索引
    bestindex, part = chooseBestFeatureToSplit2(data)
    if bestindex == -1:
        return "是"
    if type([t[bestindex] for t in data][0]) == type("a"):
        #判断是否时离散的
        bestlabel = labels[bestindex]
        Tree = {bestlabel: {}}
        temp = copy.copy(labels)
        feature = [example[bestindex] for example in data]
        #已经选择的特征不再参与分类，将该类别删除
        del (temp[bestindex])
        #该属性所有可能取值，也就是节点的分支
        unique = set(feature)
        for i in unique:  
            s = temp[:] 
            Tree[bestlabel][i] = decision_tree1(split(data, bestindex, i), s)
    else: #连续的变量
        bestlabel = labels[bestindex] + "<" + str(part)
        Tree = {bestlabel: {}}
        temp = labels[:]
        del(temp[bestindex])
        leftdata = split2(data, bestindex, part, 0)
        Tree[bestlabel]["是"] = decision_tree1(leftdata, temp)
        rightdata = split2(data, bestindex, part, 1)
        Tree[bestlabel]["否"] = decision_tree1(rightdata, temp)
    return Tree

In [61]:
tree1=decision_tree1(train2,labels2)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Seri

### 分类

In [62]:
def classify2(data, tree, labels):
    #根节点
    firstStr = list(tree.keys())[0]
    firstLabel = firstStr
    #查看是否是连续型
    t = str(firstStr).find('<')
    #如果是连续型的特征
    if t > -1:
        firstLabel = str(firstStr)[ : t]
    secondDict = tree[firstStr]
    #跟节点对应的属性
    featIndex = labels.index(firstLabel)
    result = ''
    #对每个分支循环
    for key in list(secondDict.keys()):
        if type(data[featIndex]) == type("a"):
            #测试样本进入某个分支
            if data[featIndex] == key:
                if type(secondDict[key]).__name__ == 'dict':
                    #该分支不是叶子节点，递归
                    result = classify2(data, secondDict[key], labels)
                else:
                    #如果是叶子，返回结果
                    result = secondDict[key]
        else:
            value = float(str(firstStr)[t + 1 : ])
            if data[featIndex] <= value:
                if type(secondDict['是']).__name__ == 'dict':
                    #该分支不是叶子节点，递归
                    result = classify2(data, secondDict['是'], labels)
                else:
                    #如果是叶子，返回结果
                    result = secondDict['是']
            else:
                if type(secondDict['否']).__name__ == 'dict':
                    #该分支不是叶子节点，递归
                    result = classify2(data, secondDict['否'], labels)
                else:
                    #如果是叶子，返回结果
                    result = secondDict['否']
    return result

### 预测

In [63]:
def predict1(train, test, labels):
    predictions = []
    tree = decision_tree1(train, labels)
    print("生成决策树如下:", tree)
    for index in test:
        predictions.append(classify2(index, tree, labels))
    return predictions

In [64]:
predictions1=predict1(train2,test2,labels2)
print(predictions1)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Seri

生成决策树如下: {'纹理': {'稍糊': {'敲声': {'浊响': {'密度<0.56': {'是': '是', '否': '是'}}, '沉闷': {'密度<0.6615': {'是': '是', '否': {'根蒂': {'稍蜷': '是', '蜷缩': '是'}}}}}}, '模糊': {'密度<0.29400000000000004': {'是': '是', '否': '是'}}, '清晰': {'根蒂': {'稍蜷': {'密度<0.3815': {'是': '是', '否': {'色泽': {'青绿': '是', '乌黑': '是'}}}}, '硬挺': '是', '蜷缩': {'密度<0.5820000000000001': {'是': '是', '否': {'敲声': {'浊响': {'色泽': {'青绿': '瓜', '乌黑': '瓜'}}, '沉闷': {'色泽': {'青绿': '瓜', '乌黑': '瓜'}}}}}}}}}}
['是', '是', '是', '是', '是']


C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  a = pd.value_counts(pd.DataFrame(data).iloc[:,-1],normalize=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_2692\4006420081.py:6: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Seri

In [65]:
print("C4.5分类器对于test2数据集的准确率是：", simrate(test2, predictions1))

C4.5分类器对于test2数据集的准确率是： 60.00%


可以看出，在这种情况下，ID3的准确率高于C4.5的准确率